In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pysfa import SFA

## Import data

In [ ]:
df = pd.read_csv('../data/msfa_data.csv')

## Create object

Our model can be written as,
$$
y_i = X_i \beta + u_i - v_i + \epsilon_i.
$$

where $u_i \sim N(0, \gamma)$ and $v_i \sim HN(0, \delta)$.

In [ ]:
m = df.shape[0]
s = np.sqrt(df['uhc_variance'].values)
#
x = df['physicians'].values
z = np.ones((m,1))
d = np.ones((m,1))
y = df['uhc'].values
#
ind = np.argsort(x)
x = x[ind]
y = y[ind]
#
sfa = SFA(x.reshape(m,1), z, d, s, Y=y, add_intercept_to_x=True)

In [ ]:
plt.plot(x, sfa.Y, '.')

## Add BSpline

Need to specify
* `knots`
* `degree`
* `l_linear`(linear head) and `r_linear` (linear tail)
* `bspline_mono`: curve increasing or decreasing
* `bspline_cvcv`: curve convex or concave

In [ ]:
# add splines
knots = np.array([np.min(x), 20.0, 40.0, np.max(x)])
degree = 3
sfa.addBSpline(knots, degree, r_linear=True, bspline_mono='increasing', bspline_cvcv='concave')

## Add Constraints for the Variables

* constrain `beta` be between 0 and 1, so that the curve will be between 0 and 1
* constrain `gama` to be 0, so that $u_i$ in the equation will be 0
* constrain `deta` be be positive since it represent vairance of $v_i$

In [ ]:
beta_uprior = np.array([[0.0]*sfa.k_beta, [1.0]*sfa.k_beta])
gama_uprior = np.array([[0.0]*sfa.k_gama, [0.0]*sfa.k_gama])
deta_uprior = np.array([[0.0]*sfa.k_deta, [np.inf]*sfa.k_deta])

sfa.addUPrior(beta_uprior, gama_uprior, deta_uprior)

## Fit data

In [ ]:
sfa.optimizeSFA()

In [ ]:
plt.plot(x, sfa.Y, '.')
plt.plot(x, sfa.X.dot(sfa.beta_soln))

## Estimate Random Effect

In [ ]:
# call estimateRE function
sfa.estimateRE()

In [ ]:
plt.plot(sfa.v_soln, '.')
plt.plot(sfa.u_soln, '.')

## Forcast Data Point

Extrapolate the data using `forcastData` function. Need to provide
* new `X`
* predicted `v` corresponding to the new `X`

In [ ]:
x_new = x.copy()
X_new = x_new.reshape(x.size, 1)
# pretend the predict v value to be 0.1
v_new = sfa.v_soln.copy()

In [ ]:
# call forcastData function
y_new = sfa.forcastData(X_new, v_new, add_intercept_to_x=True)

In [ ]:
plt.plot(x, sfa.Y, '.')
plt.plot(x, sfa.X.dot(sfa.beta_soln))
plt.plot(x_new, y_new, '.r')

## Trimming SFA

In [ ]:
sfa.optimizeSFAWithTrimming(int(0.9*sfa.N), stepsize=100.0, verbose=True, max_iter=20)

In [ ]:
id_outliers = np.where(sfa.w == 0.0)[0]
plt.plot(x, y, '.')
plt.plot(x, sfa.X.dot(sfa.beta_soln))
plt.plot(x[id_outliers], y[id_outliers], 'r.')

In [ ]:
np.sqrt(sfa.deta_soln)

## Post Analysis

In [ ]:
beta_sample, re_sample, y_sample, y_mean, y_negp, y_intv = predictData(sfa, include_random_effect=True, sample_size=100)

In [ ]:
plt.plot(x, y_mean)
plt.fill_between(x, y_intv[:,0], y_intv[:,1], color='gray', alpha=0.5)

In [ ]:
sfa.X.shape

In [ ]:
sfa.beta_soln.shape